In [6]:

import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

! pwd
! source /home/qcr/cgras_moveit_ws/devel/setup.bash

/home/qcr/cgras_moveit_ws/src/cgras_robot/src


In [25]:
rospy.init_node("move_group_python_interface_tutorial", anonymous=True)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_name = "manipulator"
move_group = moveit_commander.MoveGroupCommander(group_name)
# move_group.set_pose_reference_frame('world')

[ INFO] [1703035300.343103210]: Ready to take commands for planning group manipulator.


In [36]:
print(f'pose reference frame: {move_group.get_pose_reference_frame()}')
end_effector_link = move_group.get_end_effector_link()
print(f'end-effector links: {end_effector_link}')

pose reference frame: trolley
end-effector links: ee_control_link


In [29]:
link_names = robot.get_link_names()
print(link_names)

for link in link_names:
    link_pose = move_group.get_current_pose(link)
    print(f'{link}\n{link_pose.pose.position}')
    print(f'roll, pitch, yaw: {move_group.get_current_rpy(link)}')
    #print(f'{link_pose.header.frame_id}')

['trolley', 'trolley_robot_base', 'base_link', 'base', 'base_link_inertia', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'flange', 'tool0', 'enclosure', 'ee_control_link']
trolley
x: 0.0
y: 0.0
z: 0.0
roll, pitch, yaw: [0.0, -0.0, 0.0]
trolley_robot_base
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, -0.0, 0.0]
base_link
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, -0.0, 1.5707963267948963]
base
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, 0.0, -1.5707963267948968]
base_link_inertia
x: 0.0
y: 0.0
z: 1.1
roll, pitch, yaw: [0.0, 0.0, -1.5707963267948968]
shoulder_link
x: 0.0
y: 0.0
z: 1.2273
roll, pitch, yaw: [0.0, -0.0, 3.0868036732051034]
upper_arm_link
x: 0.0
y: 0.0
z: 1.2273
roll, pitch, yaw: [-1.5707963270570027, 0.6720926535897931, -0.05478898054788348]
forearm_link
x: -0.4781838977720488
y: 0.026225454999761772
z: 1.6083464347314989
roll, pitch, yaw: [1.5707963270020442, -0.14060000000000003, 3.0868036731760746]
wrist_1_link
x: 0

In [44]:
obj_pose = move_group.get_current_pose(end_effector_link)
obj_pose.pose.position.x -= 0.05
obj_pose.header.frame_id = 'trolley'
move_group.set_pose_target(obj_pose)
success = move_group.go(wait=True)
if success:
    print('Successful Pose Move Goal')
else:
    print('Failed')
move_group.stop()
move_group.clear_pose_targets()
print(f'COMPLETED: {move_group.get_current_pose(end_effector_link).pose}')

Successful Pose Move Goal
COMPLETED: position: 
  x: 0.3792278980395307
  y: 0.05634155796744547
  z: 1.2020534464956403
orientation: 
  x: -0.8944546054567342
  y: -0.44537652893094337
  z: -0.039857471097785674
  w: 0.001445078666313955


In [61]:
move_group.set_named_target('stow')
success = move_group.go(wait=True)
if success:
    print('Successful Pose Move Goal')
else:
    print('Failed')
move_group.stop()
move_group.clear_pose_targets()
print(f'COMPLETED: {move_group.get_current_pose(end_effector_link).pose}')


Successful Pose Move Goal
COMPLETED: position: 
  x: 0.3792396413869683
  y: 0.056457884559340726
  z: 1.2018805082548663
orientation: 
  x: 0.8945474303959831
  y: 0.4451596747486434
  z: 0.0402199697370636
  w: 0.00033583325563900016


In [60]:
def display_trajectory(robot, plan):
    global display_trajectory_publisher
    if 'display_trajectory_publisher' not in globals():
        display_trajectory_publisher = rospy.Publisher(
            "/move_group/display_planned_path",
            moveit_msgs.msg.DisplayTrajectory,
            queue_size=20,
        )
    display_trajectory = moveit_msgs.msg.DisplayTrajectory()
    display_trajectory.trajectory_start = robot.get_current_state()
    display_trajectory.trajectory.append(plan)
    display_trajectory_publisher.publish(display_trajectory)

waypoints = []
scale = 1
wpose = move_group.get_current_pose().pose
wpose.position.z += scale * 0.1  # First move up (z)
wpose.position.y += scale * 0.2  # and sideways (y)
waypoints.append(copy.deepcopy(wpose))

wpose.position.x += scale * 0.1  # Second move forward/backwards in (x)
waypoints.append(copy.deepcopy(wpose))

wpose.position.y -= scale * 0.1  # Third move sideways (y)
waypoints.append(copy.deepcopy(wpose))

(plan, fraction) = move_group.compute_cartesian_path(
    waypoints, 0.01, 0.0  # waypoints to follow  # eef_step # jump_threshold
)
print(f'plan: {plan}\nfraction: {fraction}')
display_trajectory(robot, plan)

success = move_group.execute(plan, wait=True)
if success:
    print('Successful Pose Move Goal')
else:
    print('Failed')

plan: joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "trolley"
  joint_names: 
    - shoulder_pan_joint
    - shoulder_lift_joint
    - elbow_joint
    - wrist_1_joint
    - wrist_2_joint
    - wrist_3_joint
  points: 
    - 
      positions: [4.657506505112397, -2.4694136087518648, 2.610082519176509, -1.7818084051799494, -1.531759934954578, 0.5938680189880542]
      velocities: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      accelerations: [0.9588724252990397, 0.0, 0.0, 0.0, 0.0, 0.0]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [4.699595583384263, -2.4459937902930937, 2.6143202338142135, -1.811042954118214, -1.534751669141112, 0.6358802538041366]
      velocities: [0.24315559677009974, 0.14224683969211452, 0.023116786027775418, -0.1739619509053617, -0.017540033724753126, 0.24267475306876565]
      accelerations: [0.9749864809298462, 0.6095064719930474, 0.08500065511708345, -0.72